In [1]:
# Here is an example of what it is possible to do using the ACADEMY methodology. 


# For more informations please read Zanetta et al,. 2018 
# Don't hesitate to contact the author via Github or directly by email.

In [3]:
# We first import the different pacakges necessary to run the whole code
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs # this version of hyperspy has merged the Pull request #1835 and #1462

In [4]:
s=hs.load("data.hspy") # We load a cropped section of the raw data of the zone 2 of the Paris meteorite presented in the paper.
s

<EDSSEMSpectrum, title: Map Sum Spectrum, dimensions: (97, 110|512)>

In [5]:
s.plot(True) # We plot the hyperspectral signal and the different X-ray lines.
             # The signal have been calibrated in energy beforehand

In [6]:
carto=hs.load('cartography.hspy')

In [25]:
s.change_dtype('float')

In [26]:
s.decomposition('nmf')

In [9]:
s.plot_cumulative_explained_variance_ratio()

In [10]:
s.blind_source_separation(8)

In [27]:
a=s.get_decomposition_loadings()

In [11]:
a=s.get_decomposition_factors()

In [28]:
a.plot()

In [38]:
d=np.ones([110,97,6])
for i in range (0,5):
    b=a.data[i]/np.max(a.data[i])
    c=b>0.90
    d[:,:,1]=c*i

In [39]:
d=hs.signals.EDSSEMSpectrum(d)

In [40]:
d.as_signal1D(1)

<EDSSEMSpectrum, title: , dimensions: (6, 97|110)>

In [41]:
d.plot()

In [33]:
len(a)

512

In [34]:
e=a.data[0]/np.max(a.data[0])

In [217]:
s.plot()

In [35]:
plt.imshow(d,cmap='jet')

TypeError: Invalid dimensions for image data

In [42]:
for i in range (0,6):
    b=a.data[i]/np.max(a.data[i])
    plt.figure()
    plt.imshow(b,cmap='jet')

In [43]:
b>0.9

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [44]:
np.shape(b)

(110, 97)

In [45]:
m=s.create_model(False,False)

In [46]:
for i in range (0,len(a)):
    g=hs.model.components1D.ScalableFixedPattern(hs.signals.EDSSEMSpectrum(a.data[i]))
    m.extend([g])

In [47]:
m.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   1 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   2 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   3 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   4 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   5 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   6 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   7 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   8 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
   9 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
  10 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
  11 | ScalableFixedPatt.. | ScalableFixedPatt.. | ScalableFixedPatt..
  12 |

In [48]:
for i in range (0,len(m)):
    m.set_parameters_not_free([i], parameter_name_list=['xscale','shift'])
    m.set_parameters_value('yscale',value=0.5,component_list=[i])

In [49]:
m.multifit(bounded=True,grad=True)

ValueError: non-broadcastable output operand with shape (512,) doesn't match the broadcast shape (110,512)

In [ ]:
m.plot(plot_components=True)

In [50]:
d.plot()

In [42]:
hs.plot.plot_spectra([sr.inav[30,7,0],d.inav[30,7,0]])

C:\Anaconda3\lib\site-packages\matplotlib\__init__.py:892: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [82]:
plt.close('all')

In [162]:
m.plot(plot_components=True)

In [ ]:
m.save('goodfit.hdf5', 'model')

In [ ]:
u=hs.load('crop2.hdf5')
m =u.models.restore('model')

In [112]:
u.metadata.Sample.elements=[]
u.add_lines

<bound method EDS_mixin.add_lines of <EDSSEMSpectrum, title: Map Sum Spectrum model from decomposition with 12 components, dimensions: (41, 22, 1|512)>>

In [113]:
u.metadata

├── Acquisition_instrument
│   └── SEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 0.0
│       │       ├── elevation_angle = 29.0
│       │       ├── energy_resolution_MnKa = 130.0
│       │       ├── live_time = 340787.1875
│       │       └── real_time = 52578.2421875
│       ├── beam_energy = 5.0
│       ├── magnification = X440
│       └── tilt_stage = 0.0
├── General
│   ├── date = 2017-01-24
│   ├── original_filename = Paris_section-6_Map2
│   ├── time = 17:42:00
│   └── title = Map Sum Spectrum model from decomposition with 12 components
├── Sample
│   ├── elements = []
│   └── xray_lines = []
└── Signal
    ├── binned = True
    ├── quantity = X-rays (Counts)
    └── signal_type = EDS_SEM

In [114]:
m=u.models.restore('model')

ValueError: Component name "Mn_La" not found in model

In [102]:
m3.components

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 |  background_order_6 |  background_order_6 |          Polynomial
   1 |               Al_Ka |               Al_Ka |            Gaussian
   2 |               Al_Kb |               Al_Kb |            Gaussian
   3 |                C_Ka |                C_Ka |            Gaussian
   4 |               Ca_Ka |               Ca_Ka |            Gaussian
   5 |               Ca_Kb |               Ca_Kb |            Gaussian
   6 |               Ca_La |               Ca_La |            Gaussian
   7 |               Ca_Ll |               Ca_Ll |            Gaussian
   8 |               Ca_Ln |               Ca_Ln |            Gaussian
   9 |               Cr_La |               Cr_La |            Gaussian
  10 |              Cr_Lb3 |              Cr_Lb3 |            Gaussian
  11 |               Cr_Ll |               Cr_Ll |            Gaussian
  12 |

In [80]:
j=d.isig[2.5:]

f=j.integrate1D('Energy')

C:\Anaconda3\lib\site-packages\matplotlib\text.py:886: ComplexWarning: Casting complex values to real discards the imaginary part
  y = float(self.convert_yunits(self._y))
C:\Anaconda3\lib\site-packages\matplotlib\text.py:755: ComplexWarning: Casting complex values to real discards the imaginary part
  posy = float(textobj.convert_yunits(textobj._y))


In [81]:
#for i in range (0,len(y)) :

b=f.data[0]
b=np.float32(b)
tiff.imsave('D:\\Users\Documents\Python Scripts\Carto_Paris_Zone1_110117\Carto tiff rebin\ ' + 'U' + '.tif',(b))

C:\Anaconda3\lib\site-packages\matplotlib\text.py:886: ComplexWarning: Casting complex values to real discards the imaginary part
  y = float(self.convert_yunits(self._y))
C:\Anaconda3\lib\site-packages\matplotlib\text.py:755: ComplexWarning: Casting complex values to real discards the imaginary part
  posy = float(textobj.convert_yunits(textobj._y))


In [29]:
plt.close('all')
for i in range (0,len(y)) :
    b=y[i].data
    plt.figure()
    plt.imshow(b[0,:,:])
    plt.title(s.metadata.Sample.xray_lines[i])
    plt.legend()

C:\Anaconda3\lib\site-packages\matplotlib\axes\_axes.py:519: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [85]:
whos

Variable   Type              Data/Info
--------------------------------------
BytesIO    type              <class '_io.BytesIO'>
Image      module            <module 'PIL.Image' from <...>packages\\PIL\\Image.py'>
cluster    module            <module 'sklearn.cluster'<...>n\\cluster\\__init__.py'>
cv2        module            <module 'cv2' from 'C:\\A<...>\cv2.cp35-win_amd64.pyd'>
d          EDSSEMSpectrum    <EDSSEMSpectrum, title: M<...>ensions: (41, 22, 1|512)>
g1         Gaussian          <Gaussian (Gaussian component)>
g2         Gaussian          <Gaussian_0 (Gaussian component)>
hs         module            <module 'hyperspy.api' fr<...>kages\\hyperspy\\api.py'>
hyperspy   module            <module 'hyperspy' from '<...>\\hyperspy\\__init__.py'>
m          EDSSEMModel       <EDSSEMModel, title: Map <...>ition with 12 components>
np         module            <module 'numpy' from 'C:\<...>ges\\numpy\\__init__.py'>
plt        module            <module 'matplotlib.pyplo<...>\\matplo